In [1]:
#Import packages 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import statistics
import math
import random
from scipy import stats
from statsmodels.graphics.mosaicplot import mosaic

In [2]:
#Import, and slice data (2nd sample group first 100 responses from revised survey, not including 1st sample group of 20)

df = pd.read_csv("UMass Sustainability Initiative.csv", sep=",")


In [3]:
#basic data analysis for quantitive data, habits and age distribution

def distribution(df):
    print(f'The mean is {df.mean()} the Standard Erorr of the mean is {stats.sem(df, axis=None, ddof=0)}')
    print(f'The variance is {statistics.variance(df)} the standard deviation is {df.std()}')
    print(f'The min is {df.min()}, Q1 is {np.quantile(df, .25)}, median is {df.median()}, Q3 is {np.quantile(df, .75)}, max is {df.max()}')
    print(f'Skewness: {stats.skew(df)}')
    first_quartile = np.quantile(df, .25)
    third_quartile = np.quantile(df, .75)
    IQR = third_quartile - first_quartile 
    low_value = first_quartile - (IQR * 1.5)
    high_value = third_quartile + (IQR * 1.5)
    outliers = []
    outliers.clear
    for i in df:
        if low_value > i:
            outliers.append(i)
        elif high_value < i:
            outliers.append(i)
    outliers.sort()
    print(f'Outliers: {outliers}')
        
#distribution(df['Habits'])



In [4]:
#Data Cleaning

df.rename(columns = {'How old are you?':'Age', 'What is your gender?':'Gender', 'What occupation do you hold in the University?':'Occupation',
                     'What year are you?':'Year', 'What is your major?':'Major',
                     'Do you live on campus?':'On Campus', 'If you do live on campus, where do live?':'Location',
                     'How often do you recycle?':'Frequency', "Why do you recycle? (Don't answer if you don't)":'Reasoning',
                     'Are the recycling rules on campus clearly presented?':'Rules', 
                     'If not, if the recycling rules were clearer would you be more likely to recycle?':'Likelihood',
                     'Do you look for the recycle symbol before recycling?':'Symbol', 'What items do you usually recycle?':'Items',
                     'Did the survey overall make you think about your own personal recycling methods and habits?':'Habits',
                     'Whats the best way to reach you for the prize draw? (Email, Phone, etc. )':'Prize'}, inplace = True)

#fill with None?
#df.fillna(None, inplace=True)

#Splitting strings into arrays, not actually useful! Strings have useful methods see electronics cell below
#df['Reasoning'] = df['Reasoning'].str.split(";", expand = False)
#df['Items'] = df['Items'].str.split(";", expand = False)

df['Reasoning'].fillna('Nothing', inplace = True) 

#Gender data cleaning...
df['Gender'].replace(to_replace = "Goblin", value = "other", inplace = True) 


#age data cleaning....
df['Age'].replace(to_replace = "Twenty", value = int(20), inplace = True) 
df['Age'].replace(dict.fromkeys(['I am banes', 'RECYCLING IS FOR NERDS'], int(20)), inplace = True)
#df['Age'].replace(to_replace = 1, value = 20, inplace = True) 

#Replacing Majors with relevant groups, by engineers, Computer Science, Business 
#Replacing Majors with relevant groups, by engineers, Computer Science, Business 
df['Major'].replace(dict.fromkeys(['CS', 'CIS', 'Computer Science', 'Computer science', 'Computer science', 'Comp sci', 'Comp Sci', 'COMPUTER SCIENCE', 'Computer Science'], 
                                  'Computer Science'), inplace = True)
df['Rules'].replace(dict.fromkeys(["I didn't know there were any recycling rules!"], "I didn't know"), inplace = True)
df['Major'].replace(dict.fromkeys(['MIS', 'Accounting', 'Marketing'], 'College of Business'), inplace = True)

#focus group 1 is now df1
df1 = df.iloc[0:100]

df.head(3)

,Timestamp,Age,Gender,Occupation,Year,Major,On Campus,Location,Frequency,Reasoning,Rules,Likelihood,Symbol,Items,Habits,"Whats the best way to reach you for the prize draw? (Enter your Email, Phone, etc. )"
0,2020/02/18 4:03:42 PM AST,20,Male,NaN,Sophomore,Computer Science,Yes,Birch,Often,Sustainability;Ease of Access,Yes,Yes,Yes,Paper;Cardboard;Plastic Bottles;Glass Bottles;...,3,NaN
1,2020/02/18 4:05:49 PM AST,20,Male,NaN,Junior,Computer Science,Yes,Aspen,Always,Sustainability;Habit,No,Yes,No,Paper;Cardboard;Plastic Bottles;Glass Bottles;...,4,jegan2@umassd.edu
2,2020/02/18 4:09:10 PM AST,23,Male,NaN,Junior,Computer Science,No,NaN,Always,Habit,Yes,NaN,No,Paper;Cardboard;Plastic Bottles;Glass Bottles;...,2,NaN


In [5]:
#CONFIDENCE INTERVAL FOR FREQUENCY OF RECYCLING
#repacling all strings with ints in NEW dataframe column
#if you want to use for the WHOLE data set, change df to whatever the name is of the dataframe
#that contains all 446 participants
df['Frequency_as_int'] = df['Frequency']    
df['Frequency_as_int'] = df['Frequency_as_int'].replace('Always', 5)
df['Frequency_as_int'] = df['Frequency_as_int'].replace('Often', 4)
df['Frequency_as_int'] = df['Frequency_as_int'].replace('Sometimes', 3)
df['Frequency_as_int'] = df['Frequency_as_int'].replace('Rarely', 2)
df['Frequency_as_int'] = df['Frequency_as_int'].replace('Never', 1)
#function: mean +- z (of alpha/2) * (std dev/ sqrt(n))
frequency_mean = df['Frequency_as_int'].mean()
frequency_std_div = df['Frequency_as_int'].std()
#using the equation we get 1-((1 - 95)/2)
#z_score is 1.96 for 95 percent confidence
z_score = 1.96
divisor = math.sqrt(len(df))
confidence = z_score * (frequency_std_div/divisor)
print('we are 95% confident that the interval', confidence+frequency_mean, 'to', frequency_mean-confidence, 'contains the population mean based on the sample set')
correlate = np.corrcoef(df1['Age'], df1['Frequency_as_int'])
print(f'Correlation: {correlate}')


we are 95% confident that the interval 4.0398538043813605 to 3.8570071821657246 contains the population mean based on the sample set


KeyError: 'Frequency_as_int'

In [ ]:
#Pick a random winner for the prize
# df[random.randint(1, len(df))], first attempt doesn't work
df.loc[df.index == random.randint(1, len(df))]


In [ ]:
# age data analysis, pain in the neck
#df['Age'].fillna(method = 'ffill')
#df.reset_index(drop=True)
#df['Age'].dtype #object type
df['Age'].value_counts()
#df1.Age = df1.Age.astype(int)
#df.Age = df.Age.astype(int)
#print(df['Age'].mean(skipna=True))

distribution(df['Age'])



In [ ]:
# habits data analysis, luckily a lot easier
#print(df['Habits'].mean(skipna=True))

distribution(df['Habits'])
df['Habits'].value_counts()


In [ ]:
#basic data analysis for qualtitve data, Major Location Frequency Reasoning Items ... etc

#to be filled in...

In [ ]:
#Sorting entries (potential group of interest could be people with low habits, majority of people had high schore for habits)

df.sort_values('Major', ascending=True)

In [ ]:
#Initial data analysis with Bar plots on recycling frequency (Focus Group Students)
#we can assume that all first 100 surveys are students since we manually surveyed students with QR codes on our phones

plt.style.use('fivethirtyeight')
x = np.arange(5)

plt.bar(x, df1['Frequency'].value_counts(), align='center', alpha=0.5)
plt.xticks(x, ('Always', 'Often', 'Sometimes', 'Rarely', 'Never'))
plt.title('Frequency of Recycling Among Students')
plt.xlabel('How often do you recycle?')
plt.ylabel('Focus Group 2 Students')
plt.show()
#cumulative frequency would be cool on the graph above, Pareto Chart

In [ ]:
#Data analysis on Items column, specifically electronic subset

AC = df.Items.str.contains(pat = 'Electronics') 
#AC.value_counts().plot(kind='bar'), same as code below without formatting

x = np.arange(2)

plt.bar(x, AC.value_counts(), align='center', alpha=0.5)
plt.xticks(x, ("Doesn't Recycle Electronics", 'Recycles Electronics'))
plt.title('Students who Recycle Electronics')
plt.xlabel('What items do you usually recycle?')
plt.ylabel('Focus Group 2 Students')
plt.show()

#df1.Items.value_counts()

In [ ]:
#mosaic plot
m = df.groupby(["Symbol", "Likelihood"]).sum()

mosaic(df, ["Habits", "Symbol"])
plt.show()

In [ ]:
#Data analysis on Reasoning column specifically how manuy people put 'Sustainability' as a reason for them recycling 

df1 = df.Reasoning.str.contains(pat = 'Sustainability')
df1.value_counts()
df1.value_counts().plot(kind='bar')
pl1 = df.Reasoning.str.contains('Sustainability').sum()

In [ ]:
#Data analysis on Reasoning column specifically how manuy people put 'Habit' as a reason for them recycling 
df2 = df.Reasoning.str.contains(pat = 'Habit')
df2.value_counts()
df2.value_counts().plot(kind='bar')
pl2 = df.Reasoning.str.contains('Habit').sum()
#print(df.Reasoning.str.contains('Habit').sum())    

In [ ]:
#Data analysis on Reasoning column specifically how manuy people put 'Ease of Access' as a reason for them recycling 
df3 = df.Reasoning.str.contains(pat = 'Ease of Access')
df3.value_counts()
df3.value_counts().plot(kind='bar')
pl3 = df.Reasoning.str.contains('Ease of Access').sum()
#print(df.Reasoning.str.contains('Ease of Access').sum())

In [ ]:
#prints graph of how many people chose what fro their reason for recycling 
#there is more than 100 answers because people could choose more than one thing, to my detriment
print(df.Reasoning.str.contains('Sustainability').sum())    

df4 = pd.DataFrame({'Reasons for Recycling':['Sustainability', 'Habit', 'Ease of Access'], 'Number of People':[pl1, pl2, pl3]}) 
ax = df4.plot.bar(x='Reasons for Recycling', y='Number of People', rot=0)

In [ ]:
#Jonas' Distaster Area 2.0
#plt.style.use('classic')
#Recycling Frequency Pie Chart

fig1, ax1 = plt.subplots()
labels = df['Frequency'].value_counts().index.tolist()

wedges, texts, autotexts = ax1.pie(df['Frequency'].value_counts(), autopct='%1.1f%%', wedgeprops={'edgecolor': 'black'}, textprops=dict(color='w'))
ax1.set_title('Recycling Frequency')

ax1.legend(loc="center right", bbox_to_anchor=(1, 0, 0.5, 1), labels= labels)
plt.setp(autotexts, size=12, weight="bold")

In [ ]:
#Gender Bar Graph
fig3, ax3 = plt.subplots()
x = df['Gender'].value_counts().index.tolist()
y = df['Gender'].value_counts().tolist()
ax3.bar(x, y, edgecolor='black')


ax3.set_axisbelow(True)
ax3.set_xlabel('Genders')
ax3.set_ylabel('Number of Students')
ax3.set_title('Number of Students per Gender')

In [ ]:
#Age Box Plot
fig4, ax4 = plt.subplots()

ax4.boxplot(df['Age'], meanline=True, showmeans=True, sym='r+')

ax4.set_axisbelow(True)
ax4.set_xlabel('')
ax4.set_ylabel('Frequency')

In [ ]:
#Rules and likeliehood Mosaic
#m = df.groupby(['Rules','Likelihood']).sum()
mosaic(df, ["Rules", "Likelihood"])


In [ ]:
#Major Bar Graph
fig5, ax5 = plt.subplots()
x = df['Major'].value_counts().index.tolist()
y = df['Major'].value_counts().tolist()
ax5.barh(x, y, edgecolor='black')


ax5.set_axisbelow(True)
ax5.set_xlabel('Majors')
ax5.set_ylabel('Number of Students')
ax5.set_title('Number of Students per Major')


plt.tight_layout()
plt.show()

In [ ]:
# Age V.S. Frequency Scatter Plot
plt.scatter(x=df['Age'],
            y=df['Habits'], alpha = 0.2)
plt.title('Age V.S. Impact')
plt.xticks(np.arange(0, 50, 10))
plt.yticks(np.arange(0, 6, 1))
plt.xlabel('Age')
plt.ylabel('Impact')
plt.show()

In [ ]:
#HAHAHAHAHA
#print(df['Reasoning'][0])
#print(len(df['Age']))

#need num of people from ages 18-25 and 25+
#need how many people slected each for each age 

sus18 = 0
hab18 = 0
ease18 = 0
sus19= 0
hab19 = 0
ease19 = 0
sus20 = 0
hab20 = 0
ease20 = 0
sus21 = 0
hab21 = 0
ease21 = 0
sus22 = 0
hab22 = 0
ease22 = 0
sus23 = 0
hab23 = 0
ease23 = 0
sus24 = 0
hab24 = 0
ease24 = 0
sus25 = 0
hab25 = 0
ease25 = 0
sus25_plus = 0
hab25_plus = 0
ease25_plus = 0

for i in range (0, len(df['Age'])):
    if df['Age'][i] == 18:
        
        print(df['Age'][i])
        print(df['Reasoning'][i])
        
        if 'Sustainability' in df['Reasoning'][i]:
            sus18+=1
        elif 'Habit' in df['Reasoning'][i]:
            hab18+=1
        elif 'Ease of Access' in df['Reasoning'][i]:
            ease18+=1
            
    elif df['Age'][i] == 19:
        
        print(df['Age'][i])
        print(df['Reasoning'][i])
        
        if 'Sustainability' in df['Reasoning'][i]:
            sus19+=1
        elif 'Habit' in df['Reasoning'][i]:
            hab19+=1
        elif 'Ease of Access' in df['Reasoning'][i]:
            ease19+=1
            
    elif df['Age'][i] == 20:
        if 'Sustainability' in df['Reasoning'][i]:
            sus20+=1
        elif 'Habit' in df['Reasoning'][i]:
            hab20+=1
        elif 'Ease of Access' in df['Reasoning'][i]:
            ease20+=1
            
    elif df['Age'][i] == 21:
        
        print(df['Age'][i])
        print(df['Reasoning'][i])
        print(i)
        
        if 'Sustainability' in df['Reasoning'][i]:
            sus21+=1
        elif 'Habit' in df['Reasoning'][i]:
            hab21+=1
        elif 'Ease of Access' in df['Reasoning'][i]:
            ease21+=1
        
    elif df['Age'][i] == 22:
        if 'Sustainability' in df['Reasoning'][i]:
            sus22+=1
        elif 'Habit' in df['Reasoning'][i]:
            hab22+=1
        elif 'Ease of Access' in df['Reasoning'][i]:
            ease22+=1
        
    elif df['Age'][i] == 23:
        if 'Sustainability' in df['Reasoning'][i]:
            sus23+=1
        elif 'Habit' in df['Reasoning'][i]:
            hab23+=1
        elif 'Ease of Access' in df['Reasoning'][i]:
            ease23+=1
        
    elif df['Age'][i] == 24:
        if 'Sustainability' in df['Reasoning'][i]:
            sus24+=1
        elif 'Habit' in df['Reasoning'][i]:
            hab24+=1
        elif 'Ease of Access' in df['Reasoning'][i]:
            ease24+=1
        
    elif df['Age'][i] == 25:
        if 'Sustainability' in df['Reasoning'][i]:
            sus25+=1
        elif 'Habit' in df['Reasoning'][i]:
            hab25+=1
        elif 'Ease of Access' in df['Reasoning'][i]:
            ease25+=1
        
    elif df['Age'][i] > 25:
        if 'Sustainability' in df['Reasoning'][i]:
            sus25_plus+=1
        elif 'Habit' in df['Reasoning'][i]:
            hab25_plus+=1
        elif 'Ease of Access' in df['Reasoning'][i]:
            ease25_plus+=1
        
       
x_labels = ['Sustainaility', 'Habit', 'Ease of Access']

y_units_18 = [sus18, hab18, ease18]
y_units_19 = [sus19, hab19, ease19]
y_units_20 = [sus20, hab20, ease20]
y_units_21 = [sus21, hab21, ease21]
y_units_22 = [sus22, hab22, ease22]
y_units_23 = [sus23, hab23, ease23]
y_units_24 = [sus24, hab24, ease24]
y_units_25 = [sus25, hab25, ease25]
y_units_greater_25 = [sus25_plus, hab25_plus, ease25_plus]

plt.plot(y_units_18, color='green')
plt.plot(y_units_19, color='red')
plt.plot(y_units_20, color='orange')
plt.plot(y_units_21, color='purple')
plt.plot(y_units_22, color='blue')
plt.plot(y_units_23, color='black')
plt.plot(y_units_24, color='pink')
plt.plot(y_units_25, color='brown')
plt.plot(y_units_greater_25, color='yellow')

plt.title("Reasoning for Recycling Based on Age")

plt.xlabel('Reasons for Recycling')
plt.ylabel('Number of People')
plt.xticks(np.arange(len(x_labels)), x_labels, rotation=0)

green_line = mpatches.Patch(color='green', label='18')
red_line = mpatches.Patch(color='red', label='19')
orange_line = mpatches.Patch(color='orange', label='20')
purple_line = mpatches.Patch(color='purple', label='21')
blue_line = mpatches.Patch(color='blue', label='22')
black_line = mpatches.Patch(color='black', label='23')
pink_line = mpatches.Patch(color='pink', label='24')
brown_line = mpatches.Patch(color='brown', label='25')
yellow_line = mpatches.Patch(color='yellow', label='>25')

plt.legend(handles=[green_line, red_line, orange_line, purple_line, blue_line, black_line, pink_line,brown_line, yellow_line])

plt.grid(True)
           
plt.show()

print("test")

In [ ]:
#Gender Pie Chart
fig6, ax6 = plt.subplots()
labels = df1['Gender'].value_counts().index.tolist()

wedges, texts, autotexts = ax6.pie(df1['Gender'].value_counts(), autopct='%1.1f%%', wedgeprops={'edgecolor': 'black'}, textprops=dict(color='w'))
ax6.set_title('Number of Students per Gender')

ax6.legend(loc="center right", bbox_to_anchor=(1, 0, 0.5, 1), labels= labels)
plt.setp(autotexts, size=12, weight="bold")


In [ ]:
#year Pie Chart
fig7, ax7 = plt.subplots()
labels = df1['Year'].value_counts().index.tolist()

wedges, texts, autotexts = ax7.pie(df1['Year'].value_counts(), autopct='%1.1f%%', wedgeprops={'edgecolor': 'black'}, textprops=dict(color='w'))
ax7.set_title('Number of Students per Year')

ax7.legend(loc="center right", bbox_to_anchor=(1, 0, 0.5, 1), labels= labels)
plt.setp(autotexts, size=12, weight="bold")

In [ ]:
#Age v. Frequency Scatter plot
fig8, ax8 = plt.subplots()

ax8.scatter(df['Frequency_as_int'], df['Age'], marker='D', s=10, alpha=.5)
x_axis = ["","Never", "Rarely", "Sometimes", "Often", "Always"]
ax8.set_xticklabels(x_axis)
ax8.set_title('Recycling Frequency v. Age')
ax8.set_xlabel('Recycling Frequency')
ax8.set_ylabel('Age')